# Competition — Network Generation

### Challenge Overview

Your goal is to generate a network that is as close as possible to the original real network. You do not have the original network in the explicit view, but you know some of its statistics. All statistics are in [`stats.txt`](https://drive.google.com/file/d/1xifvP4hVIHm6t6u2fdV6ZPNPC3euHMu1/view?usp=sharing) filethat contains a dictionary of the form
* number_nodes (number of nodes): value
* number_cc (number of connected components): value, sigma
* radius (radius of giant component): value, sigma
* diameter (diameter of giant component): value, sigma
* average_clustering (average clustering coefficient): value, sigma
* average_path_length (average path length): value, sigma
* degree_cdf (empirical CDF of degree distribution): values, probabilities

Meaning of all these sigmas is described in Evaluation section.

You can use this code to draw CDF
```python
q_seq, p_seq = stats['degree_cdf']
    plt.plot(
        np.append(np.repeat(q_seq, 2)[1:], q_seq[-1]),
        np.repeat(p_seq, 2)
    )
    plt.show()
```

### Evaluation Criteria

Your total score is calculated as weighted sum of 6 scores — similarities between statistics of original and generated networks. Each score takes values from the interval [0, 1], where 1 — absolute similarity with the original network. The scores are
* "KS"
    * 1 - KS_dist
    * where KS_dist is Kolmogorov-Smirnov test statistic value
* "Radius"
    * $\text{GK}(r, r', \sigma_r) = \exp\left[-\frac{(r - r')^2}{2\sigma_r^2}\right]$
    * where GK is Gaussian Kernel, $r$ is a radius of the original network, $r'$ is a radius of a generated network, $\sigma_r$ is a sigma of a radius from `stats.txt` file
* "Diameter", "Av. clustering", "Av. path length", "Number of CC" are calculated by Gaussian Kernel in the same way
* "Total"
    * 1/6 KS + 1/6 Radius + 1/6 Diameter + 1/6 Av. clustering + 1/6 Av. path length + 1/6 Number of CC

All scores immediately take value 0 if a generated network has incorrect number of nodes.

**Baselines**

Baselines 1-2 are calculated by the following algorithm:
1. Generate a random degree sequence using Inverse Transform Sampling
2. Generate a valid graph by Configuration Model
3. Calculate total score
4. Repeat 1-3 steps 1000 times and accumulate a set of total scores

**Grades:**
1. $total ≥ 0.1$
2. $total ≥ 0.12$
3. $total ≥ 0.15$
4. $total ≥ 0.189$  (beat a mean total score)
5.  $total ≥ 0.279$ (beat a mean + 3*sigma total score)
6. $total ≥ 0.4$
7. $total ≥ 0.486$
8.  $total ≥ 0.6$
9. $total ≥ 0.85$
10. $total ≥ 0.95$



### Submission Guidelines

Submit a .ipynb file that generates the graph from scratch.

In [1]:
! gdown 1xifvP4hVIHm6t6u2fdV6ZPNPC3euHMu1

Downloading...
From: https://drive.google.com/uc?id=1xifvP4hVIHm6t6u2fdV6ZPNPC3euHMu1
To: /content/stats.txt
100% 694/694 [00:00<00:00, 2.17MB/s]


In [2]:
! cat stats.txt

{"number_nodes": 1882, "radius": [15, 2], "diameter": [28, 4], "average_clustering": [0.005066798238955518, 0.001], "average_path_length": [11.748410823170731, 2], "number_cc": [168, 32], "degree_cdf": [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 21, 24, 46], [0.0, 0.6902231668437833, 0.8517534537725824, 0.9086078639744952, 0.9378320935175345, 0.9516471838469713, 0.9654622741764081, 0.9723698193411264, 0.9776833156216791, 0.9808714133900106, 0.9845908607863975, 0.9888416578108395, 0.9893730074388948, 0.9925611052072264, 0.9936238044633369, 0.9952178533475027, 0.9957492029755579, 0.9968119022316685, 0.997874601487779, 0.9989373007438895, 0.9994686503719448, 1.0]]}

In [14]:
import numpy as np
import networkx as nx

In [15]:
# # G = nx.erdos_renyi_graph(1882, 0.001, seed=42)

# # G = nx.erdos_renyi_graph(34, 0.3, seed=42)
# # G = nx.cycle_graph(34)
# G = nx.path_graph(30)
# # print(G.nodes)
# for i in range(168):
#     H = nx.path_graph(11)
#     # H = nx.erdos_renyi_graph(11, 0.1, seed=42)
#     # H = nx.cycle_graph(11)
#     nx.relabel_nodes(H, {j:(30 + i*11 + j) for j in range(11)}, copy=False)
#     G = nx.compose(G, H)

# H = nx.path_graph(4)
# nx.relabel_nodes(H, {j:(1878 + j) for j in range(4)}, copy=False)
# G = nx.compose(G, H)


# largest_cc = G.subgraph(max(nx.connected_components(G))).copy()

In [16]:
# # G = nx.erdos_renyi_graph(1882, 0.001, seed=42)

# # G = nx.erdos_renyi_graph(34, 0.3, seed=42)
# # G = nx.cycle_graph(34)
# G = nx.path_graph(30)
# # print(G.nodes)
# for i in range(166):
#     H = nx.path_graph(11)
#     nx.relabel_nodes(H, {j:(30 + i*11 + j) for j in range(11)}, copy=False)
#     G = nx.compose(G, H)

# for i in range(166, 168):
#     H = nx.erdos_renyi_graph(11, 0.37, seed=42)
#     nx.relabel_nodes(H, {j:(30 + i*11 + j) for j in range(11)}, copy=False)
#     G = nx.compose(G, H)

# H = nx.path_graph(4)
# nx.relabel_nodes(H, {j:(1878 + j) for j in range(4)}, copy=False)
# G = nx.compose(G, H)


# largest_cc = G.subgraph(max(nx.connected_components(G))).copy()

# Final answer is the cell below

In [17]:
G = nx.path_graph(34)
G.add_edge(1, 8)

for i in range(136):
    H = nx.star_graph(10)
    nx.relabel_nodes(H, {j:(34 + i*11 + j) for j in range(11)}, copy=False)
    G = nx.compose(G, H)

for i in range(136, 166):
    H = nx.path_graph(11)
    nx.relabel_nodes(H, {j:(34 + i*11 + j) for j in range(11)}, copy=False)
    G = nx.compose(G, H)

for i in range(166, 168):
    H = nx.erdos_renyi_graph(11, 0.37, seed=42)
    nx.relabel_nodes(H, {j:(34 + i*11 + j) for j in range(11)}, copy=False)
    G = nx.compose(G, H)

In [18]:
nx.write_edgelist(G, "answer.edgelist", data=False)

# Some checks

In [19]:
largest_cc = G.subgraph(max(nx.connected_components(G))).copy()
print(f'Radius: {nx.radius(largest_cc)}, should be 15')
print(f'Diameter: {nx.diameter(largest_cc)}, should be 28')
print(f'Average path length: {nx.average_shortest_path_length(largest_cc)}, should be 11.7')

print(f'Number of cc: {nx.number_connected_components(G)}')
print(f'Number of nodes in cc: {nx.number_of_nodes(largest_cc)}')
print(f'Number of nodes: {nx.number_of_nodes(G)}')
print(f'Average clustering: {nx.average_clustering(G)}')

Radius: 15, should be 15
Diameter: 29, should be 28
Average path length: 10.807486631016042, should be 11.7
Number of cc: 169
Number of nodes in cc: 34
Number of nodes: 1882
Average clustering: 0.004893477050756542


In [20]:
degree_seq = [d for n, d in G.degree]
np.mean(degree_seq)

1.853347502656748

In [21]:
evaluate(G, 'stats.txt')

{'ks_distance': 0.934643995749203,
 'radius': 1.0,
 'diameter': 0.9692332344763441,
 'average_clustering': 0.9850921221682316,
 'average_path_length': 0.8952365518174149,
 'number_cc': 0.9995118379398894,
 'total': 0.9639529570251804}

### Code for evaluation

The following code takes the graph G and path to the `stats.txt` file and computes all the scores.  

In [10]:
import numpy as np
import networkx as nx

from scipy.stats import ks_2samp

def evaluate(G, stats_file='stats.txt'):
    def gaussian_kde(l, r, sigma):
      return np.exp(-(l - r)**2 / (2 * sigma**2))

    def flatten(xss):
      return [x for xs in xss for x in xs]

    with open(stats_file, 'r') as file:
        stats = eval(file.read())

    #Incorrect number of nodes
    if G.number_of_nodes() != stats['number_nodes']:
        return {'kstest': 0,
                'radius': 0,
                'diameter': 0,
                'average_clustering': 0,
                'average_path_length': 0,
                'number_cc': 0}
    #Compute metrics
    largest_cc = G.subgraph(max(nx.connected_components(G))).copy()
    solution = {}
    solution['radius'] = nx.radius(largest_cc)
    solution['diameter'] = nx.diameter(largest_cc)
    solution['average_clustering'] = nx.average_clustering(G)
    solution['average_path_length'] = nx.average_shortest_path_length(largest_cc)
    solution['number_cc'] = nx.number_connected_components(G)
    degree_seq = [d for n, d in G.degree]

    #Compare  scores
    res = {}
    stats['degree_seq'] = flatten([[deg] * count for deg, count in zip(stats['degree_cdf'][0][1:], np.round(np.diff(stats['degree_cdf'][1]) * stats['number_nodes']).astype('int'))] )
    # print(np.unique(stats['degree_seq'], return_counts=True)) ####################################
    # print(np.unique(degree_seq, return_counts=True)) ####################################
    res['ks_distance'] = 1 - ks_2samp(degree_seq, stats['degree_seq']).statistic
    for k in solution:
        res[k] = gaussian_kde(solution[k], stats[k][0], stats[k][1])

    #Compute final score
    res['total'] = sum([res[k] for k in res]) / 6
    return res

In [ ]:
G = nx.path_graph(100)
evaluate(G, 'stats.txt')

{'kstest': 0,
 'radius': 0,
 'diameter': 0,
 'average_clustering': 0,
 'average_path_length': 0,
 'number_cc': 0}

In [ ]:
G = nx.path_graph(1882)
evaluate(G, 'stats.txt')

{'ks_distance': 0.3108395324123273,
 'radius': 0.0,
 'diameter': 0.0,
 'average_clustering': 2.6625607850027013e-06,
 'average_path_length': 0.0,
 'number_cc': 1.2187610094869082e-06,
 'total': 0.05180723562235364}